## Banco de Dados com SQLite
---

aqui, é importante saber que:

campo: refere-se a menor unidade de informação como um nome, um número, etc.
***
registro: refere-se à informação total que se tem ao preencher diversos campos.
***
tabela: refere-se a unidade em que se divide os tipos de registro

#### criando um arquivo de banco de dado com SQLite
---

In [1]:
import sqlite3

após importá-lo, é necessáro criar o banco de dado, como no exemplo:
```
bd = sqlite3.connect('agenda.db')
```

no exemplo, o arquivo do banco de dados chama-se `agenda` e a extensão db refere-se a database mas é mais uma convensão do que de fato uma necessidade de pô-lo no nome do arquivo, só é recomendado que ele seja diferente de uma extensão real como pdf, py, txt, etc.

após criar o banco de dados, é necessário criar um cursor.
```
cursor = bd.cursor()
```
Cursores são objetos utilizados para enviar comandos e receber resultados dos bancos de dados, sendo possível manipulá-los através dos cursores.

a primeira manipulação pode ser a criação duma tabela neste banco que vai armazenar nome e telefone de alguém ou alguma empresa, por exemplo. para isto, é necessário o uso do cursor:
```
cursor.execute('create table agenda(nome text, telefone text)')
```
o comando que cria a tabela é o `create table`, sendo necessário passar o nome da tabela, neste caso: `agenda`. em seguida, é necessário passar os campos desta tabela, neste caso: nome e telefone seguidos de seus tipos (text, que, no python, equivale a string).

com a tabela criada, para inserir informações nos campos da tabela é necessário:
```
cursor.execute('insert into agenda(nome, telefone) values(?, ?)', ('Lucas', '(84) 9.8747-4615'))
```
assim, o comando `insert` precisa do nome da tabela e de seus campos passados entre parêntesis, neste caso: `agenda(nome, telefone)`. o `value(?, ?)` é o campo que será preenchido com a tupla que vem logo em seguida, substituindo `?` pelas informações respectivas.

para fazer vários registros ao mesmo tempo é necessário mudar o `execute` por `executemany` sendo todas as informações passadas por tuplas dentro de uma tupla ou lista principal:
```
cursor.executemany('insert into agenda(nome, telefone) values(?, ?)', [('Lucas', '(84) 9.8747-4615'), ('Pai', '(84) 9.8884-4785')])
```

para que as mudanças sejam salvas, é necessário:
```
bd.commit()
```
e, em seguida, se nada mais for ser feito neste banco, é preciso fechar tanto o cursor quando o banco de dados:
```
cursor.close()
bd.close()
```

pode criar um banco de dados usando `with`, porém, é necessário importar a biblioteca `contextlib` e usar o método `closing()` para que o banco e o curso sejam encerrados automaticamente ao fim do programa:
```
from contextlib import closing
import sqlite3

with sqlite3.connect('agenda.db') as conexão:
    with closing(conexão.cursor()) as cursor()
        cursor.execute('create table agenda(nome text, telefone text)')
        cursor.execute('insert into <tabela>(nome, telefone) values(?, ?)', ('Lucas', '(84) 9.8747-4615'))
        cursor.commit()
```
da mesma forma, se for mais de um registro gravado, pode mudar normalmete `execute()` para `executemany()`.

#### lendo um arquivo de banco de dados com SQLite
---

para isto, além de ser necessário importar novamente a biblioteca `sqlite3`, deve ser criado um banco de dados e um cursor da mesma forma:
```
bd = saqlite.connect('agenda.db')
cursor = bd.cursor()
```
no caso, o arquivo passado, no exemplo: `agenda`, tem que ser o que se deseja ler.

em seguida, deve-se fazer:
```
cursor.execute('select * from agenda')
```
que basicamente está mandando selecionar __tudo__ da tabela `agenda`.

 em seguida, as informações da tabela serão transformadas em uma tupla:
```
info = cursor.fetchone()
```
sendo a variável `info` uma tupla contendo em cada um de seus índices as informações presente em cada campo:
```
print(info[0], info[1])
>>> Lucas (84) 9.8747-4615
```

se houver mais de um registro nesta tabela, para lê-la é necssário:
```
info = cursor.fetchall()
```
usando o `fetchall()` ao invés do `fetchone()`

sendo que a variável `info` agora é uma lista com várias tuplas dentro. então, para lê-la:
```
print(info[0][0], info[0][1])
>>> Lucas (84) 9. 8747-4615

print(info[1][0], info[1][1])
>>> pai (84) 9.8884-4785
```
ou pode ser usado um laço

se não for mais usar este banco de dados, é necessário fechá-lo junto do cursor:
```
cursor.close()
bd.close()
```

outra forma de ler um banco de dados usando o `with`, mas para isso é necessário importar a biblioteca `contextlib` e usar seu método `closing` pois ele encerra automaticamente o banco e o cursor ao fim do with:
```
from contextlib import closing
import sqlite3

with sqlite3.connect('agenda.db') as conexão:
    with closing(conexão.cursor()) as cursor():
        cursor.execute('select * from agenda')
        lendo = cursor.fetchone()
        print(lendo[0], lendo[1])
```
pode mudar para `fetchall()` também, usando um laço para lê-lo, se for o caso.

#### lendo um arquivo de banco de dado como um dicionário
---

observe:
```
with sqlite3.connect('agenda.db') as conexão:
    conexão.row_factory = sqlite3.Row
    with closing(conexão.cursor()) as cursor:
        for i in cursor.execute('select * from agenda'):
            print(f'{i["nome"]} {i["telefone"]}')
>>> Lucas (84) 9.1234-6789
```
observe que o comando `conexão.row_factory = sqlite3.Row` é o responsável por transformar as informações de lista para dicionário.

#### consultando e fazendo buscas nos registros
---

já tendo importado as bibliotecas `sqlite3` e `contextlib`, faremos:
```
nm = input('nome> ')

with sqlite3.connect('agenda.db') as conexão:
    with closing(conexão.cursor()) as cursor():
        cursor.execute(f'select * from agenda where nome ="{nm}"')
        rst = cursor.fetchall()
        print(rst)
```
no comando `select * from agenda where nome ="Lucas"` quer dizer que somente os campos `nome` preenchidos com o nome passado pelo usuário através da variável `nm` serão selecionados e a variável `rst` será, da mesma forma, uma lista com várias tuplas contendo os registros encontrados.

uma vulnerabilidade deste método conhecida como `SQLInjection` é que, se na variável `nm` for colocado: `X" or "1"="1`, a lista inteira vai aparecer.

#### atualizando registros
---

deve-se usar o comando como no exmplo: `cursor.execute('update agenda set telefone = "(84) 9.1234-6789") where nome = "Lucas"')`.

se todos os registros devem passar a conter a mesma informção, faça: `cursor.execute('uptade agenda set telefone = "(84) 9.0000-0000"')` sem selecionar um registro em específico.

se quiser atualizar vários registros, se não todos, mais cada um com diferentes informações, faça como o exemplo:
```
neo_dados = [("121", "fulano"), ("123", "sicrano")]
cursor.executemany('update agenda set telefone = ? where nome = ?', neo_dados)
```

fazendo `cursor.rowcount` mostra a quantidade de modificações que foram feitas no banco de dados.

fazendo `conexão.rollback()` reverte as modificações, funcionando como o inverso do `conexão.commit()`

#### alterando a tabela
---

usando o comando `cursor.execute("alter table agenda add idade interger")` um novo campo, neste caso `idade`, será adicionada, por exemplo. `integer` é o tipo do campo `idade`.

supondo que foi criado também um campo `cidade` junto do novo campo `idade`, para atualizar a tabela, deve-se fzaer:
```
dados = [22, "mossoró", "Lucas"]
cursor.executemany('update agenda set idade = ?, cidade = ? where nome = ?', dados)
```
observe que deve ser usado o `executemany()` e, usando uma vírgula entre os campos, mais de um campo pode ser atualizado de uma só vez.

#### apagando registros
---

use o `cursor.execute('delete from agenda where nome = "Lucas"')`, por exemplo, para apagar o registro de `Lucas`.

se desejar apagar todas as informações que o banco possui: `cursor.execute('delete from agenda')`.

#### chave de identificação para os registros
---

usando o comando: `cursor.execute('id integer primary key autoincrement, nome text, telefone text')`, será criada uma chave aleatória de identificação para cada registro feito, que pode ser acessado pelo índice zero ou 'id', caso esteja usando como dicionário.

é bastante útil, pois evita erros com dados repetitivos.

#### ordenando os dados
---

para ordenar em ordem alfabética: `cursor.execute('select * from agenda ordem by nome')`, ou em ordem contrária à alfabética: `cursor.execute('select * from agenda order by name desc')`

para ordem em ordem numérica crescente: `cursor.execute('select * from agenda ordem by telefone')`, ou, em ordem decrescente: `cursor.execute('select * from agenda ordem by telefone desc')`, se `telefone` um número.

também, é possível ordenar pela chave primária: `cursor.execute('select * from agenda ordem by id')`, por exemplo.

#### agrupando dados
---

com o comando `select` é possível grupos e extrair informações deste grupos usando funções como `count()`, `min()`, `max()`, `avg()` e `sum()`, por exmplo.